In [66]:
# noinspection JupyterPackage
from collections import Counter
import envs.twitter as credentials
import datetime
import tweepy
import random
import sys
import json
import pandas as pd

#set seed
random.seed(789)
random_min=20
random_max=100
hashtags=[
    "VacunaCOVID19",
    "sismo",
    "WandaVision",
    "UKlockdown3",
    "Indonesia",
    "iPhone",
    "CES2021",
    "Adictosdigitales",
    "Tech",
    "innovation",
    "iphone",
    "EE.UU"]
days_ago=5
#default date_until to today
date_until=datetime.date.today()
#default date_since to N (days_ago) from today
date_since=date_until - datetime.timedelta(days=days_ago)
exclude_filter="retweets"

In [67]:
#generate random number, arguments used as limits
def random_generator(min_arg=random_min,max_arg=random_max):
    try:
        random_result = random.randint(min_arg,max_arg)
        print(random_result)
        return random_result
    except Exception:
        print("Oops!", sys.exc_info()[0], "occurred.")

tweets_needed=random_generator()

81


In [68]:
def get_tweets(
        hashtag_arg,
        count_arg=100,
        tweets_arg=tweets_needed,
        date_since_arg=date_since,
        date_until_arg=date_until,
        exclude_filter_arg=exclude_filter
):
    try:
        # Authentication object creation - read only
        auth = tweepy.AppAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
        # API object creation
        api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
        # Cursor search with filters
        tweets = tweepy.Cursor(api.search,
            q="#{} since:{} until:{} -filter:{}".format(hashtag_arg,date_since_arg,date_until_arg,exclude_filter_arg),
            count=count_arg
        ).items(tweets_needed)
        return tweets
    except Exception:
        print("Oops!", sys.exc_info()[0], "occurred.")

In [69]:
#extract tweet specific information argument should be tweet object
def extract_tweet(tweet_arg):
    extract=[
        tweet_arg.id,
        tweet_arg.created_at.strftime('%m/%d/%Y'),
        tweet_arg.favorite_count,
        tweet_arg.retweet_count,
        tweet_arg.lang,
        tweet_arg.text,
        tweet_arg.author.id,
        tweet_arg.author.name,
        tweet_arg.author.screen_name,
        tweet_arg.author.verified,
        tweet_arg.author.followers_count
    ]
    return extract

In [70]:
# extract required information, argument should be tweepy.models.SearchResults object
def extract_info(results_arg):
    extract_result=[]
    for tweet in results_arg:
       extract_result.append(extract_tweet(tweet))
    return extract_result

In [71]:
#create dictionary of tweets iterating thru all hastags in hastags list
dictOfTweets = { i : extract_info(get_tweets(i,date_since_arg="2021-02-02",date_until_arg="2021-02-07")) for i in hashtags }

In [72]:
#function that creates dataframes with specific cols, df based on hashtag. argument should be string with no #
def df_create(hashtag_arg):
    return pd.DataFrame(dictOfTweets[hashtag_arg],
                  columns = [
                      'id', 'date_dreated','favorite','retweet','language','content',
                      'author_id','author_name','author_screen_name','verified','followers'
                  ]
                  )

1.	Propuesta de cómo medir la influencia de un usuario de la muestra.(2 puntos)


La propuesta para medir la influencia:
Influencia = estimado_impportancia_retweets+estimado_importancia_favoritos+estimado_cantidad_tweets

In [73]:
def influence_calculator(df_arg):
    df_arg['tweets_estimate'] = df_arg.groupby(['author_id'])['followers'].transform('count')/100
    df_arg["favorite_estimate"]=(10*df_arg["favorite"])/df_arg["favorite"].max()
    df_arg["retweet_estimate"]=(10*df_arg["retweet"])/df_arg["retweet"].max()
    df_arg["influence"]=df_arg["tweets_estimate"]+df_arg["favorite_estimate"]+df_arg["retweet_estimate"]
    df_arg.sort_values(by=['influence'], inplace=True,ascending=False)
    influence=df_arg[["id","author_screen_name","influence"]].head(5)
    return influence

In [74]:
#funcion para establecer la actividad de un hashtag en base a la fecha de los tweets recolectados, argumento= dataframe
def day_activity(df_arg):
    activity = df_arg.pivot_table(index=['date_dreated'], aggfunc='size')
    df_days = pd.DataFrame(activity, columns = ['count'])
    df_days.sort_values(by=['count'], inplace=True,ascending=False)
    return df_days

In [75]:
# creacion individual de dataframes para cada hashtag
df_VacunaCOVID19= df_create("VacunaCOVID19")
df_sismo= df_create("sismo")
df_WandaVision= df_create("WandaVision")
df_UKlockdown3= df_create("UKlockdown3")
df_Indonesia= df_create("Indonesia")
df_iPhone= df_create("iPhone")
df_CES2021= df_create("CES2021")
df_Adictosdigitales= df_create("Adictosdigitales")
df_Tech= df_create("Tech")
df_innovation= df_create("innovation")
df_iphone= df_create("iphone")
df_EEUU= df_create("EE.UU")

2.	Utilizando la propuesta hecha procede a ranquear al top cinco de usuarios más influyentes.  (3 puntos)
3.	Visualiza la actividad por día de cada uno de los hashtags, indicando el día de mayor actividad.   (5 puntos)

In [76]:
#mostrar rankings
def hashtag_info(hashtag_arg):
    print("Top cinco de usuarios más influyentes para #{}".format(hashtag_arg))
    print("")
    ic=influence_calculator(df_create(hashtag_arg))
    print(ic)
    print("")
    print("Actividad por día de cada para #{}".format(hashtag_arg))
    print("")
    da=day_activity(df_create(hashtag_arg))
    print(da)
    print("")
    print("Dia con mayor actividad:")
    print(da.head(1))
    print("*****************************************************************")

for hashtag in hashtags:
    hashtag_info(hashtag)

Top cinco de usuarios más influyentes para #VacunaCOVID19

                     id author_screen_name  influence
72  1358140563120402432         sanidadgob  20.010000
33  1358179051488305153            dmacuna  14.217317
40  1358172562417618946      rochimonterom   6.142791
76  1358134868543160327     SaludPublicaEs   5.276938
19  1358188772614156289           CMP_PERU   2.584526

Actividad por día de cada para #VacunaCOVID19

              count
date_dreated       
02/06/2021       81

Dia con mayor actividad:
              count
date_dreated       
02/06/2021       81
*****************************************************************
Top cinco de usuarios más influyentes para #sismo

                     id author_screen_name  influence
63  1358162532867989515         onemichile  20.010000
20  1358195078326595585            ASMXBOT   5.157059
56  1358163102257283073     ChileAlertaApp   4.708824
76  1358162224183980033      SismoDetector   3.442353
57  1358163096427245569      SismoDe

Combina las muestras de cada uno de los hashtags en un dataset global.

In [77]:
#Concatenar dataframes
df_global = pd.concat([
    df_VacunaCOVID19, df_sismo, df_WandaVision,
    df_UKlockdown3, df_Indonesia, df_iPhone,
    df_CES2021, df_Adictosdigitales, df_Tech,
    df_innovation, df_iphone, df_EEUU,
], ignore_index=True, sort=False)

4.	Crea un ranking del top 5 de los hashtags más utilizados.   (3 puntos)

In [78]:
all_hashtags=[]
for row in df_global['content']:
    all_hashtags.extend([i  for i in row.split() if i.startswith("#") ])
hashtag_count = pd.DataFrame(all_hashtags,columns = ['hashtags'])
hasthtag_activity = hashtag_count.pivot_table(index=['hashtags'], aggfunc='size')
df_hash_count = pd.DataFrame(hasthtag_activity, columns = ['count'])
df_hash_count.sort_values(by=['count'], inplace=True,ascending=False)
df_hash_count.head(5)

,count
hashtags,
#iPhone,79
#WandaVision,65
#Tech,46
#VacunaCOVID19,37
#Techno,34


5.	Crea un ranking del top 5 de usuarios con mayor número de followers.  (3 puntos)

In [79]:
df_global_user=df_global
df_global_user=df_global_user.drop_duplicates(subset=['author_id'])
df_global_user.sort_values(by=['followers'], inplace=False,ascending=False)
df_global_user[["id","author_screen_name","followers"]].head(5)


,id,author_screen_name,followers
0,1358203591601893383,ZELL_tw,61
1,1358203082824241152,peruenlanoticia,55161
2,1358201833852968962,franciscoweber,690
3,1358201306251464704,AbstractoNL,831
4,1358199742057107457,LilianaAmeiro,433


6.	Crea un ranking del top 3 de idiomas más utilizados.  (2 puntos)

In [80]:
df_global_language = df_global.pivot_table(index=['language'], aggfunc='size')
df_language = pd.DataFrame(df_global_language, columns = ['count'])
df_language.sort_values(by=['count'], inplace=True,ascending=False)
df_language.head(3)

,count
language,
en,324
es,241
ja,98


7.	Visualizar el porcentaje de usuarios verificados totales versus los no verificados  (2 puntos)

In [81]:
df_global_verified = df_global.pivot_table(index=['verified'], aggfunc='size')
df_verified = pd.DataFrame(df_global_verified, columns = ['count'])
df_verified['%']=round((df_verified['count']*100)/len(df_global),2)
df_verified.sort_values(by=['count'], inplace=True,ascending=False)
df_verified

,count,%
verified,,
False,775,94.74
True,43,5.26
